In [ ]:
# 选择对应的文本切分器，如果是通用文本的话，建议选择RecursiveCharacterTextSplitter，代码如下：
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 导入文本
loader = UnstructuredFileLoader("./data/news_test.txt")
# 将文本转成 Document 对象
data = loader.load()
print(f'documents:{len(data)}')

# 初始化加载器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(data)
print("split_docs size:",len(split_docs))

# chunk_size：每一个分片的最大大小

# chunk_overlap：相邻的块之间的最大重叠。有一些重叠可以很好地保持块之间的一些连续性（类似于一个滑动窗口）。

#生成embedding

In [ ]:
# 直接使用OpenAIEmbeddings来生成Embedding数据
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import VectorDBQA
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.embeddings.openai import OpenAIEmbeddings
import IPython
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")

embeddings = OpenAIEmbeddings( )

In [ ]:
# 使用HuggingFaceEmbeddings来生成Embedding数据
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import IPython
import sentence_transformers


embedding_model_dict = {
    "ernie-tiny": "nghuyong/ernie-3.0-nano-zh",
    "ernie-base": "nghuyong/ernie-3.0-base-zh",
    "text2vec": "GanymedeNil/text2vec-large-chinese",
    "text2vec2":"uer/sbert-base-chinese-nli",
    "text2vec3":"shibing624/text2vec-base-chinese",
}

EMBEDDING_MODEL = "text2vec3"
# 初始化 hugginFace 的 embeddings 对象
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_dict[EMBEDDING_MODEL], )
embeddings.client = sentence_transformers.SentenceTransformer(
        embeddings.model_name, device='mps')

两种方式的差别：

OpenAIEmbeddings：

使用简单，并且效果比较好；

会消耗openai的token，特别是大段文本时，消耗的token还不少，如果知识库是比较固定的，可以考虑将每次生成的embedding做持久化，这样就不需要再调用openai了，可以大大节约token的消耗；

可能会有数据泄露的风险，如果是一些高度私密的数据，不建议直接调用。

HuggingFaceEmbeddings：

可以在HuggingFace上面选择各种sentence-similarity模型来进行实验，数据都是在本机上进行计算

需要一定的硬件支持，最好是有GPU支持，不然生成数据可能会非常慢

生成的向量效果可能不是很好，并且HuggingFace上的中文向量模型不是很多。

In [ ]:
# 保存Embedding数据
# 有多种向量数据库选择，这里选择比较简单的Chroma,因为比较轻量，直接安装库就可使用。
from langchain.vectorstores import Chroma
# 初始化加载器
db = Chroma.from_documents(split_docs, embeddings,persist_directory="./chroma/openai/news_test")
# 持久化
db.persist()

# 持久化后，可以直接选择从持久化文件中加载，不需要再重新就可使用了，使用方式如下：
db = Chroma(persist_directory="./chroma/news_test", embedding_function=embeddings)


In [ ]:
# 如果只想使用embedding来计算句子的相似度，可以直接类似下面这样使用：
# 初始化 prompt 对象
question = "2022年腾讯营收多少"

similarDocs = db.similarity_search(question, include_metadata=True,k=4)

[print(x) for x in similarDocs]

# 最多返回匹配的前4条相似度最高的句子

In [ ]:
# 如果想接入ChatGLM来帮忙做总结和汇总的话，可以如下使用：
from langchain.chains import RetrievalQA
import IPython

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=ChatGLM(temperature=0.1), chain_type="stuff", retriever=retriever)
# 进行问答
query = "2022年腾讯营收多少"
print(qa.run(query))


# 本文使用LangChain+ChatGLM来实现一个简单的基于本地知识库的问答系统，可以在完全不使用openai提供的api来完成本地知识库问答系统的搭建，
# 如果有需要做本地私有化知识库问答系统的